# MLflow Demo

In [23]:
import mlflow
import numpy as np

In [ ]:
# Connect to the local MLflow server and create an experiment.
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("/demo-experiment")

## Define a model

In [25]:
class OutlierModel(mlflow.pyfunc.PythonModel):
    """Demo model that detects outliers."""

    def __init__(self, deviation=2):
        self._deviation = deviation

        # Learned parameters
        self.mean_ = None
        self.std_ = None

    def fit(self, X, y=None):
        """Fit the model to training dataset X."""
        self.mean_ = X.mean()
        self.std_ = X.std()
        return self

    def predict(self, context, X):
        """Predict whether X is an outlier."""
        lower = self.mean_ - self._deviation * self.std_
        upper = self.mean_ + self._deviation * self.std_

        return (X < lower) | (X > upper)

## Train the model

In [ ]:
# Create random training data.
train = np.random.randint(50, 100, 10)
train

In [27]:
# Create and train a local model.
local_model = OutlierModel(deviation=1)
local_model = local_model.fit(train)

In [ ]:
# Metrics learned by the local model
print(f"""
Model learned metrics:
- mean:  {local_model.mean_:.2f}
- std:   {local_model.std_:.2f}
""")

In [ ]:
# Log and store the model to MLflow.
with mlflow.start_run():

    # Create model and log parameters
    deviation = 1
    model = OutlierModel(deviation)
    mlflow.log_params({"deviation": deviation})

    # Train the model.
    model.fit(train)

    # Log the learned model metrics.
    mlflow.log_metric("mean", model.mean_)
    mlflow.log_metric("std", model.std_)

    # Log the trained model as artifact.
    model_info = mlflow.pyfunc.log_model(
        python_model=model,
        artifact_path="demo-experiment",
    )

## Register the model

In [ ]:
# URI for the model trained in the last run
model_info.model_uri

In [ ]:
# Register the model from the last run.
registered = mlflow.register_model(
    model_uri=model_info.model_uri,
    name="OutlierModel",
    tags={"release-status": "candidate"}
)

## Use the model

In [32]:
# Define a fixed test set.
test = np.array([40, 50, 60, 70, 80, 90, 100, 110])

In [ ]:
# Use the local model to validate.
local_model.predict(None, test)

In [ ]:
# Define which model and version to fetch.
model = "OutlierModel"
version = 1

# Load the MLflow model.
model_uri = f"models:/{model}/{version}"
mlflow_model = mlflow.pyfunc.load_model(model_uri)

In [ ]:
# Generate predictions.
mlflow_model.predict(test)